In [27]:
import cv2
import time
import torch
import json
import os
from datetime import datetime
from facenet_pytorch import MTCNN


In [7]:
import cv2
import time
import torch
import json
import os
import tqdm
import smtplib
from datetime import datetime
from email.message import EmailMessage
from facenet_pytorch import MTCNN


In [ ]:


SENDER_EMAIL = 'puravgguptain@gmail.com'
SENDER_PASSWORD = 'fmgs yewx lupu nmlo'
RECEIVER_EMAIL = 'vishalverma.dnm@gmail.com'


In [ ]:

def send_bulk_email_with_snapshots(snapshot_info):
    if not snapshot_info:
        print("✅ No SPAM ALERTS. No email sent.")
        return

    msg = EmailMessage()
    msg['Subject'] = f'SPAM ALERT - {len(snapshot_info)} snapshot(s) of missing eyes'
    msg['From'] = SENDER_EMAIL
    msg['To'] = RECEIVER_EMAIL

    body = "You are feeling drowsy, go to sleep.:\n\n"
    for snap in snapshot_info:
        body += f"- {snap['timestamp']} ({os.path.basename(snap['path'])})\n"

    msg.set_content(body)

    for snap in snapshot_info:
        with open(snap['path'], 'rb') as img_file:
            img_data = img_file.read()
            msg.add_attachment(
                img_data,
                maintype='image',
                subtype='jpeg',
                filename=os.path.basename(snap['path'])
            )

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
        smtp.login(SENDER_EMAIL, SENDER_PASSWORD)
        smtp.send_message(msg)
        print(f" Email sent with {len(snapshot_info)} snapshot(s)")


In [ ]:

def run_eye_detection_only():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mtcnn = MTCNN(keep_all=True, device=device)

    video_capture = cv2.VideoCapture(0)
    frame_delay = 1 / 10
    duration_sec = 10
    frame_count = 0
    log_data = []
    snapshots_to_send = []

    os.makedirs("spam_snaps", exist_ok=True)
    start_time = time.time()

    while time.time() - start_time < duration_sec:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame_count += 1
        timestamp = datetime.now()
        timestamp_str = timestamp.strftime("%Y-%m-%d %H:%M:%S")
        filename_time = timestamp.strftime("%Y%m%d_%H%M%S")

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        boxes, probs, landmarks = mtcnn.detect(rgb_frame, landmarks=True)

        eyes_detected = False

        if landmarks is not None:
            for landmark_set in landmarks:
                if landmark_set[0] is not None and landmark_set[1] is not None:
                    eyes_detected = True
                    left_eye = landmark_set[0]
                    right_eye = landmark_set[1]
                    cv2.circle(frame, (int(left_eye[0]), int(left_eye[1])), 3, (255, 0, 0), -1)
                    cv2.circle(frame, (int(right_eye[0]), int(right_eye[1])), 3, (255, 0, 0), -1)

        if eyes_detected:
            print("Eyes detected")
            log_data.append({
                "frame": frame_count,
                "timestamp": timestamp_str,
                "status": "Eyes detected"
            })
        else:
            print("Eyes not detected\nSPAM ALERT")
            log_data.append({
                "frame": frame_count,
                "timestamp": timestamp_str,
                "status": "Eyes not detected",
                "alert": "SPAM ALERT"
            })
            snap_filename = f"spam_snaps/spam_alert_{filename_time}.jpg"
            cv2.imwrite(snap_filename, frame)
            snapshots_to_send.append({"path": snap_filename, "timestamp": timestamp_str})
            print(f"> Snapshot saved: {snap_filename}")

        cv2.imshow('Eye Detection (MTCNN)', frame)
        time.sleep(frame_delay)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

    # Save log to JSON
    with open('eye_detection_log.json', 'w') as f:
        json.dump(log_data, f, indent=4)
    print("\n Log saved as eye_detection_log.json")

    # Send all snapshots via email
    send_bulk_email_with_snapshots(snapshots_to_send)




In [11]:
if __name__ == "__main__":
    run_eye_detection_only()


Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043025.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043025.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043026.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043026.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043027.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043027.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043028.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043028.jpg


2025-04-12 04:30:28.974 python[37231:61608265] _TIPropertyValueIsValid called with 16 on nil context!
2025-04-12 04:30:28.974 python[37231:61608265] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-04-12 04:30:28.974 python[37231:61608265] Text input context does not respond to _valueForTIProperty:


Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043029.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043029.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043029.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043030.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043030.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043031.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043031.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043031.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043032.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043032.jpg
Eyes not detected
SPAM ALERT
> Snapshot saved: spam_snaps/spam_alert_20250412_043032.jpg
Eyes not detected
SPA